## Dataset preaparation

#### Import necessary packages

In [910]:
import numpy as np
import pandas as pd
import os

#### Import Data form csv

In [911]:
text=os.getenv('TEXT_DATA')
rates=os.getenv('RATES_DATA')

In [912]:
# Import files
text_df=pd.read_csv(text)
rates_df=pd.read_csv(rates)

#### Check the data

In [913]:
text_df.head()

,Unnamed: 0,Date,Type,Text
0,0,20230412,0,The Federal Reserve on Wednesday released the ...
1,1,20230412,0,The minutes for each regularly scheduled meeti...
2,2,20230412,0,The minutes can be viewed on the Board's website.
3,3,20230412,0,"For media inquiries, e-mail [email protected] ..."
4,4,20230412,0,Minutes of the Federal Open Market Committee\r...


In [914]:
rates_df.head()

,Release Date,Time,Actual,Forecast,Previous
0,"Nov 01, 2023",13:00,5.50%,5.50%,5.50%
1,"Sep 20, 2023",13:00,5.50%,5.50%,5.50%
2,"Jul 26, 2023",13:00,5.50%,5.50%,5.25%
3,"Jun 14, 2023",13:00,5.25%,5.25%,5.25%
4,"May 03, 2023",13:00,5.25%,5.25%,5.00%


In [915]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9974 entries, 0 to 9973
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  9974 non-null   int64 
 1   Date        9974 non-null   int64 
 2   Type        9974 non-null   int64 
 3   Text        9974 non-null   object
dtypes: int64(3), object(1)
memory usage: 311.8+ KB


In [916]:
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Release Date  201 non-null    object
 1   Time          201 non-null    object
 2   Actual        201 non-null    object
 3   Forecast      201 non-null    object
 4   Previous      201 non-null    object
dtypes: object(5)
memory usage: 8.0+ KB


#### Remove useless columns and format relevant ones

In [917]:
columns_to_keep = ['Release Date', 'Actual']
rates_df=rates_df[columns_to_keep]
rates_df=rates_df.rename(columns = {'Release Date' : 'date'})
rates_df=rates_df.rename(columns={'Actual': 'rate'})

In [918]:
rates_df['date'] = rates_df['date'].astype(str)

In [919]:
rates_df.head()

,date,rate
0,"Nov 01, 2023",5.50%
1,"Sep 20, 2023",5.50%
2,"Jul 26, 2023",5.50%
3,"Jun 14, 2023",5.25%
4,"May 03, 2023",5.25%


Conversion to datetime

In [920]:
rates_df['date'] = pd.to_datetime(rates_df['date'], format = '%b %d, %Y')

In [921]:
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    201 non-null    datetime64[ns]
 1   rate    201 non-null    object        
dtypes: datetime64[ns](1), object(1)
memory usage: 3.3+ KB


In [922]:
# ensure rates are number object
rates_df['rate']=rates_df['rate'].str.strip('%').astype(float)

In [923]:
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 201 entries, 0 to 200
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    201 non-null    datetime64[ns]
 1   rate    201 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 3.3 KB


In [924]:
rates_df.head()

,date,rate
0,2023-11-01,5.50
1,2023-09-20,5.50
2,2023-07-26,5.50
3,2023-06-14,5.25
4,2023-05-03,5.25


In [925]:
text_df['Date']=pd.to_datetime(text_df['Date'], format='%Y%m%d')
text_df=text_df.rename(columns={'Date': 'date'})
text_df=text_df.rename(columns={'Text': 'text'})
text_df=text_df.rename(columns={'Type': 'type'})

In [926]:
# Eliminate redundant rates decisions for which we don't have text
start_date_text_df = text_df['date'].min()
rates_df = rates_df[rates_df['date'] >= start_date_text_df]

In [927]:
rates_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 72 entries, 0 to 71
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    72 non-null     datetime64[ns]
 1   rate    72 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 1.7 KB


#### Convert type to text

In [928]:
# convert classification to number
text_df['type_text'] = text_df['type'].apply(lambda x: 'statement' if x == 0 else 'minutes')

In [929]:
text_df.drop(['Unnamed: 0','type'], axis=1, inplace=True)

In [930]:
text_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9974 entries, 0 to 9973
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       9974 non-null   datetime64[ns]
 1   text       9974 non-null   object        
 2   type_text  9974 non-null   object        
dtypes: datetime64[ns](1), object(2)
memory usage: 233.9+ KB


#### Convert rates to text

In [931]:
# convert numbers into up or down
rates_df['rate_change'] = rates_df['rate'].diff()
rates_df['rate_change_text'] = rates_df['rate_change'].apply(lambda x: 'up' if x > 0 else ('down' if x < 0 else 'no change')).astype(str)

In [932]:
rates_df.head()

,date,rate,rate_change,rate_change_text
0,2023-11-01,5.50,NaN,no change
1,2023-09-20,5.50,0.00,no change
2,2023-07-26,5.50,0.00,no change
3,2023-06-14,5.25,-0.25,down
4,2023-05-03,5.25,0.00,no change


## Data Preprocessing

In [933]:
from datetime import timedelta

from sklearn.preprocessing import OrdinalEncoder

### Sliding window technique

To avoid losing information that may be inherent to the sequentiality of the text, use sliding window technique to
add all text relevant to one rate decision in a row. apply this only to press releases. keep minutes as a separate window


In [934]:
# sort data in cronological order 
rates_df.sort_values(by='date', inplace=True)
text_df.sort_values(by='date', inplace=True)

In [935]:
# Calculate the difference between consecutive rate decisions to determine dynamic window size
rates_df['next_date'] = rates_df['date'].shift(-1)
rates_df['date_diff'] = (rates_df['next_date'] - rates_df['date']).dt.days

In [936]:
rates_df.head()

,date,rate,rate_change,rate_change_text,next_date,date_diff
71,2015-01-28,0.25,0.0,no change,2015-03-18,49.0
70,2015-03-18,0.25,0.0,no change,2015-04-29,42.0
69,2015-04-29,0.25,0.0,no change,2015-06-17,49.0
68,2015-06-17,0.25,0.0,no change,2015-07-29,42.0
67,2015-07-29,0.25,0.0,no change,2015-09-17,50.0


In [937]:
# Check for NaNs since we are subtracting time deltas
print(rates_df[pd.isna(rates_df['date_diff'])])

        date  rate  rate_change rate_change_text next_date  date_diff
0 2023-11-01   5.5          NaN        no change       NaT        NaN


In [938]:
rates_df['date_diff'] = rates_df['date_diff'].fillna(0).astype(int)

In [939]:
print(rates_df[pd.isna(rates_df['date_diff'])])

Empty DataFrame
Columns: [date, rate, rate_change, rate_change_text, next_date, date_diff]
Index: []


In [940]:
# isolate statements and minutes as they occurr at different times relative to previous decisions
statement_df = text_df[text_df['type_text'] == 'statement']
minutes_df = text_df[text_df['type_text'] == 'minutes']    

In [941]:
statement_df.info()
minutes_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1729 entries, 9973 to 0
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       1729 non-null   datetime64[ns]
 1   text       1729 non-null   object        
 2   type_text  1729 non-null   object        
dtypes: datetime64[ns](1), object(2)
memory usage: 54.0+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 8245 entries, 9815 to 96
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       8245 non-null   datetime64[ns]
 1   text       8245 non-null   object        
 2   type_text  8245 non-null   object        
dtypes: datetime64[ns](1), object(2)
memory usage: 257.7+ KB


In [942]:
# group texts for each rate decision with a dynamic window size
def group_text(rate_date, text_df, date_diff):
    window_size = timedelta(days=date_diff)

    # Filter texts that occurred before the rate decision
    valid_texts = text_df[text_df['date'] < rate_date]

    # Apply sliding window: Get texts within the specified window size before rate_date
    texts_in_window = valid_texts[valid_texts['date'] >= rate_date - window_size]

    # Combine the texts
    grouped_texts = ' '.join(texts_in_window['text'])
    
    return grouped_texts


In [943]:
pairing_data  = []

for _, rate_row in rates_df.iterrows():
    rate_date = rate_row['date']
    rate = rate_row['rate_change_text']
    date_diff = rate_row['date_diff']

    grouped_statements = group_text(rate_date, statement_df, date_diff)
    grouped_minutes = group_text(rate_date, minutes_df, date_diff)
    
    # Add the data to pairing_df
    pairing_data.append({
        'decision': rate,
        'date': rate_date,
        'grouped_statements': grouped_statements,
        'grouped_minutes': grouped_minutes,
        'window_size_days': date_diff  # Store the dynamic window size for reference
    })

    
    # Convert pairing_df to a DataFrame
    pairing_df = pd.DataFrame(pairing_data)    

In [944]:
pairing_df = pairing_df.set_index("date")

In [945]:
pairing_df= pairing_df.drop(columns=['window_size_days'])

In [946]:
pairing_df.head()

,decision,grouped_statements,grouped_minutes
date,,,
2015-01-28,no change,"\r\n For media inquiries, call 202-452-2...",
2015-03-18,no change,\nSubmission of Tender\nParticipants must subm...,
2015-04-29,no change,\r\n Information received since the Fede...,"Michael Dotsey, Craig S. Hakkio, Evan F. Koeni..."
2015-06-17,no change,\r\n The Federal Open Market Committee o...,
2015-07-29,no change,\r\n Voting for the FOMC monetary policy ...,"Glenn Follette and Paul A. Smith, Assistant Di..."


#### One-hot-encode

In [947]:
print(pairing_df['decision'].value_counts())

decision
no change    47
down         20
up            5
Name: count, dtype: int64


In [948]:
ordinal_encoder = OrdinalEncoder()

In [949]:
ordinal_encoder.fit(pairing_df[["decision"]])
display(ordinal_encoder.categories_)

[array(['down', 'no change', 'up'], dtype=object)]

In [950]:
pairing_df["decision_encoded"] = ordinal_encoder.transform(pairing_df[["decision"]]).astype(int)

In [951]:
print(pairing_df['decision_encoded'].value_counts())

decision_encoded
1    47
0    20
2     5
Name: count, dtype: int64


In [952]:
pairing_df.head()

,decision,grouped_statements,grouped_minutes,decision_encoded
date,,,,
2015-01-28,no change,"\r\n For media inquiries, call 202-452-2...",,1
2015-03-18,no change,\nSubmission of Tender\nParticipants must subm...,,1
2015-04-29,no change,\r\n Information received since the Fede...,"Michael Dotsey, Craig S. Hakkio, Evan F. Koeni...",1
2015-06-17,no change,\r\n The Federal Open Market Committee o...,,1
2015-07-29,no change,\r\n Voting for the FOMC monetary policy ...,"Glenn Follette and Paul A. Smith, Assistant Di...",1


## Deep Learning Model

The model of choice are FinBERT for feature extraction (transforming text into numerical embeddings) and BiLSTM for sequence modeling and prediction

FinBERT is a domain-specific NLP trained on financial data. BiLSTM works well with sequences; it can recognize patterns over time

In [953]:
import torch
from transformers import AutoTokenizer, AutoModel, pipeline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

import torch.nn as nn
import torch.optim as optim

from sklearn.metrics import accuracy_score

#### Vectorization (FinBERT)

No preprocessing for symbols, stopwords or spaces are needed. FinBERT is a pre-trained vectorizer that knows how to handle domain-specific raw text

In [954]:
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModel.from_pretrained("ProsusAI/finbert")

In [955]:
# vectorization using FinBERT

def get_finbert_embedding(text):
    # Ensure text is not empty or NaN
    if not text or text == "nan" or pd.isna(text):
        return np.zeros((768,))  # Return a zero-vector if input is empty or NaN
    
    # Proceed with tokenization and embedding generation
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    
    return outputs.last_hidden_state[:, 0, :].numpy()  

In [956]:
pairing_df['statement_embedding'] = pairing_df['grouped_statements'].apply(lambda x: get_finbert_embedding(str(x)))
pairing_df['minutes_embedding'] = pairing_df['grouped_minutes'].apply(lambda x: get_finbert_embedding(str(x)))

In [957]:
final_df=pairing_df[['decision_encoded','statement_embedding','minutes_embedding']]

In [958]:
final_df.head(1)

,decision_encoded,statement_embedding,minutes_embedding
date,,,
2015-01-28,1,"[[-0.24246012, 0.67152435, -0.20416914, -0.289...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


#### Modeling (BiLSTM)

Since BilSTM works with sequential data, we need to merge the embeddings, standardize lenght and then splito into test and training set

In [959]:
# since we are dealing with NaNs in minutes_embedding, we add a conditionality when stacking the two arrays
# this makes sure the size of the array is the same even if we generat a 1D 0 array for NaNs previously
final_df['combined_embedding'] = final_df.apply(
    lambda row: np.hstack((row['statement_embedding'].squeeze(), 
                           (row['minutes_embedding'].squeeze() if isinstance(row['minutes_embedding'], np.ndarray) else np.zeros(768)))), 
    axis=1
)

/tmp/ipykernel_84098/4280570965.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df['combined_embedding'] = final_df.apply(


In [960]:
final_df= final_df.drop(columns=['statement_embedding', 'minutes_embedding'])

In [961]:
final_df.head(1)

,decision_encoded,combined_embedding
date,,
2015-01-28,1,"[-0.24246011674404144, 0.6715243458747864, -0...."


In [962]:
# convert the columns to np arays
X = np.vstack(final_df['combined_embedding'].values)
y = final_df['decision_encoded'].values

In [963]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Initialize Model

In [964]:
# we build the model through a Class function because it's easier to separate tasks and use the model modularly
# we also store various statistics this way within class comands so we do not have to worry about it

class BiLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(BiLSTM, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Multiply by 2 for bidirectional
        self.relu = nn.ReLU()

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        last_out = lstm_out[:, -1, :]  # Take the last time step's output
        return self.fc(self.relu(last_out))

In [965]:
def initialize_model(input_dim, hidden_dim, output_dim):
    """
    Initializes the BiLSTM model.
    Arguments:
    - input_dim: Dimensionality of the input features.
    - hidden_dim: Dimensionality of the hidden layers.
    - output_dim: Number of classes for classification.
    
    Returns:
    - model: An instance of the BiLSTM model.
    """
    model = BiLSTM(input_dim, hidden_dim, output_dim)
    return model

In [966]:
# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # Ensure correct shape

In [967]:
print(X_train_tensor.shape)
print(y_train_tensor.shape)

torch.Size([57, 1536])
torch.Size([57])


In [968]:
# BiLSTM will need 3D tensors. our tensors only have 2Ds for X
# conversely, only 1D tensor for y with integer data, and not float
X_train_tensor = X_train_tensor.unsqueeze(1)

In [969]:
print(X_train_tensor.shape)
print(y_train_tensor.shape)

torch.Size([57, 1, 1536])
torch.Size([57])


Compile Model

In [1]:
# Count occurrences in y_train
class_counts = y_train.value_counts().sort_index().values  # Ensure ordering is correct

# Compute class weights
class_weights = torch.tensor([1 / count for count in class_counts], dtype=torch.float32)

print("Class Counts (Train Set Only):", class_counts)
print("Class Weights:", class_weights)

NameError: name 'y_train' is not defined

In [972]:
# Parameters for optimization (lstm and fc) are saved by the class with which we generate model so no need to 
# specify again
def compile_model(model, learning_rate=0.001):
    """
    Compiles the model by defining the loss function and optimizer.
    Arguments:
    - model: The model to compile.
    - learning_rate: Learning rate for the optimizer.
    
    Returns:
    - criterion: The loss function (CrossEntropyLoss by default).
    - optimizer: The optimizer (Adam).
    """
    criterion = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    return criterion, optimizer

#### Training

In [973]:
def evaluate_model(model, X_test_tensor, y_test_tensor):
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # Disable gradient computation for efficiency
        outputs = model(X_test_tensor)  # Get the model's output

        logits = outputs  # These are the raw predictions (logits)

        # Get the predicted class (highest logit)
        predicted_labels = torch.argmax(logits, dim=1)
        
    # Compute accuracy by comparing predicted labels to true labels
    accuracy = accuracy_score(y_test_tensor.numpy(), predicted_labels.numpy())
    return accuracy

In [974]:
# Train model using k-fold
def train_model(X, y, input_dim, hidden_dim, output_dim, k):
    fold_accuracies =[]
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
        print(f"Training fold {fold + 1}/{k}...")

        # Split data into training and validation sets for this fold
        X_train_fold, X_val_fold = X[train_idx], X[val_idx]
        y_train_fold, y_val_fold = y[train_idx], y[val_idx]

        # Convert to PyTorch tensors
        X_train_fold_tensor = torch.tensor(X_train_fold, dtype=torch.float32)
        y_train_fold_tensor = torch.tensor(y_train_fold, dtype=torch.long)
        X_val_fold_tensor = torch.tensor(X_val_fold, dtype=torch.float32)
        y_val_fold_tensor = torch.tensor(y_val_fold, dtype=torch.long)

        # Initialize the model, criterion, and optimizer
        model = initialize_model(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim)
        criterion, optimizer = compile_model(model)

        # Train the model on this fold
        model.train()  # Set model to training mode
        for epoch in range(epochs):
            optimizer.zero_grad()
            outputs = model(X_train_fold_tensor)
            loss = criterion(outputs, y_train_fold_tensor)
            loss.backward()
            optimizer.step()

        # Evaluate the model on the validation set
        accuracy = evaluate_model(model, X_val_fold_tensor, y_val_fold_tensor)
        print(f"Fold {fold + 1} Accuracy: {accuracy:.4f}")

        # Save the accuracy for this fold
        fold_accuracies.append(accuracy)

    # Calculate average accuracy across all folds
    average_accuracy = np.mean(fold_accuracies)
    print(f"\nAverage Cross-Validation Accuracy: {average_accuracy:.4f}")

    return model, average_accuracy

In [975]:
# Model parameters
input_dim = X_train.shape[1]  # Same as embedding size
hidden_dim = 128
output_dim = 3  # For multiclass classification (up, down, no change)

In [976]:
# due to the very small size of the dataset we will use k-fold CV
k = 5 
kf = KFold(n_splits=k, shuffle=True, random_state=42)

In [977]:
# hyperparameters 
learning_rate = 0.001

In [978]:
model, average_accuracy = train_model(X_train_tensor, y_train_tensor, input_dim, hidden_dim, output_dim, k)

/tmp/ipykernel_84098/191328093.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_fold_tensor = torch.tensor(X_train_fold, dtype=torch.float32)
/tmp/ipykernel_84098/191328093.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_fold_tensor = torch.tensor(y_train_fold, dtype=torch.long)
/tmp/ipykernel_84098/191328093.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_fold_tensor = torch.tensor(X_val_fold, dtype=torch.float32)
/tmp/ipykernel_84098/191328093.py:16: UserWarning: To copy construct from a tensor, it is

Training fold 1/5...
Fold 1 Accuracy: 0.4167
Training fold 2/5...


/tmp/ipykernel_84098/191328093.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_fold_tensor = torch.tensor(X_train_fold, dtype=torch.float32)
/tmp/ipykernel_84098/191328093.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_fold_tensor = torch.tensor(y_train_fold, dtype=torch.long)
/tmp/ipykernel_84098/191328093.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_fold_tensor = torch.tensor(X_val_fold, dtype=torch.float32)
/tmp/ipykernel_84098/191328093.py:16: UserWarning: To copy construct from a tensor, it is

Fold 2 Accuracy: 0.6667
Training fold 3/5...


/tmp/ipykernel_84098/191328093.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_fold_tensor = torch.tensor(X_train_fold, dtype=torch.float32)
/tmp/ipykernel_84098/191328093.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_fold_tensor = torch.tensor(y_train_fold, dtype=torch.long)
/tmp/ipykernel_84098/191328093.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_fold_tensor = torch.tensor(X_val_fold, dtype=torch.float32)
/tmp/ipykernel_84098/191328093.py:16: UserWarning: To copy construct from a tensor, it is

Fold 3 Accuracy: 0.7273
Training fold 4/5...


/tmp/ipykernel_84098/191328093.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_fold_tensor = torch.tensor(X_train_fold, dtype=torch.float32)
/tmp/ipykernel_84098/191328093.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_fold_tensor = torch.tensor(y_train_fold, dtype=torch.long)
/tmp/ipykernel_84098/191328093.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_fold_tensor = torch.tensor(X_val_fold, dtype=torch.float32)
/tmp/ipykernel_84098/191328093.py:16: UserWarning: To copy construct from a tensor, it is

Fold 4 Accuracy: 0.5455
Training fold 5/5...


/tmp/ipykernel_84098/191328093.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train_fold_tensor = torch.tensor(X_train_fold, dtype=torch.float32)
/tmp/ipykernel_84098/191328093.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_fold_tensor = torch.tensor(y_train_fold, dtype=torch.long)
/tmp/ipykernel_84098/191328093.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_val_fold_tensor = torch.tensor(X_val_fold, dtype=torch.float32)
/tmp/ipykernel_84098/191328093.py:16: UserWarning: To copy construct from a tensor, it is

Fold 5 Accuracy: 0.5455

Average Cross-Validation Accuracy: 0.5803


In [984]:
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [985]:
print(X_test_tensor.shape)
print(y_test_tensor.shape)

torch.Size([15, 1, 1536])
torch.Size([15])


In [986]:
outputs = model(X_test_tensor)

In [987]:
accuracy = evaluate_model(model, X_test_tensor, y_test_tensor)
print(f"Test Set Accuracy: {accuracy:.4f}")

Test Set Accuracy: 0.8000


In [1]:
# The model quality will be limited by the few instances of dependent variable available, and by their skewness. There are only 5 cases of up against 40 hold decisions.
# Weighting classes while compiling the model and using cross validation can at time mitigate these issues but more observatrions are needed to improve the model performance